In [596]:
# import sys
# !pip install --prefix {sys.prefix} ipywidgets

In [621]:
from ipywidgets import Text, ToggleButton, ToggleButtons, FloatSlider
from ipywidgets import IntSlider, Layout, Button, Box, AppLayout, IntProgress

#### Nagłówek: konfiguracja

In [599]:
confHeader = ToggleButton(
    description = 'Konfiguracja',
    disabled = False,
    button_style = 'danger', 
    layout = Layout(width = '72%', height = '50px')
)

#### Slidery: wielkość populacji, ilość bitów, liczba epok

In [600]:
populationSl = HBox([
    Label(
        'Liczba osobników populacji', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 200, step = 1, value = 20, 
        layout = Layout(width = '50%'),
    )
])

bitsSl = HBox([
    Label(
        'Liczba bitów w chromosomie', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 20, step = 1, value = 8, 
        layout = Layout(width = '50%'),
    )
])

epochSl = HBox([
    Label(
        'Liczba epok', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 500, step = 1, value = 100, 
        layout = Layout(width = '50%'),
    )
])

In [601]:
confSliders = Box(
    children = [populationSl, bitsSl, epochSl], 
    layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
)
configLayout = AppLayout(header = confHeader, center = confSliders)

#### Nagłówek: operatory genetyczne

In [602]:
genHeader = ToggleButton(
    description = 'Operatory genetyczne',
    disabled = False,
    button_style = 'danger', 
    layout = Layout(width = '72%', height = '50px')
)

#### Przyciski: selekcja, krzyżowanie, mutacja

In [603]:
selectionBtn = ToggleButtons(
    options = ['najlepszych', 'kołem ruletki', 'turniejowa', 'strategia elitarna'],
    description = 'Selekcja: ',
    button_style = 'danger',
)

crossoverBtn = ToggleButtons(
    options = ['jednopunktowe', 'dwupunktowe', 'trzypunktowe', 'jednorodne'],
    description = 'Krzyżowanie: ',
    button_style = 'warning',
)

mutationBtn = ToggleButtons(
    options = ['jednopunktowa', 'dwupunktowa', 'brzegowa'],
    description = 'Mutacja: ',
    button_style = 'success',
)

#### Slidery: selekcja, krzyżowanie, mutacja

In [604]:
selectionSl = HBox([
    Label(
        'Procent osobników', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 100, step = 1, value = 30, 
        layout = Layout(width = '50%'), 
    )
])

crossoverSl = HBox([
    Label(
        'Prawdopodobieństwo krzyżowania', 
        layout = Layout(width = '22%')), 
    FloatSlider(
        min = 0, max = 1, step = 0.1, value = 0.9, 
        layout = Layout(width = '50%'),
    )
])

mutationSl = HBox([
    Label(
        'Prawdopodobieństwo mutacji', 
        layout = Layout(width = '22%')), 
    FloatSlider(
        min = 0, max = 1, step = 0.1, value = 0.2, 
        layout = Layout(width = '50%'),
    )
])

#### Layout: operatory genetyczne

In [605]:
buttons = Box(
    children = [selectionBtn, crossoverBtn, mutationBtn], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)
geneticSliders = Box(
    children = [selectionSl, crossoverSl, mutationSl], 
    layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
)
box = Box(
    children = [buttons, secondSliders], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

geneticLayout = AppLayout(header = genHeader, center = box)

In [606]:
startButton = ToggleButton(
    description = 'Rozpocznij',
    disabled = False,
    button_style = 'success', 
    layout = Layout(width = '72%', height = '50px')
)

In [607]:
app = Box(
    children = [configLayout, geneticLayout, startButton], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

In [608]:
app

Box(children=(AppLayout(children=(ToggleButton(value=False, button_style='danger', description='Konfiguracja',…

In [609]:
epochProgress = IntProgress(
    value = 7,
    min = 0,
    max = 10,
    step = 1,
    description = 'Epoka:',
    bar_style = '',
)

epochProgress

IntProgress(value=7, description='Epoka:', max=10)

#### Rezultaty: czas obliczeń, buttony do wyświetlenia wykresów, zapis wyników do pliku

In [760]:
timeOutput = HBox([
    Label(
        'Czas obliczeń: ', 
        layout = Layout(width = '100px')), 
    Label(
        '00.00 sek', 
        layout = Layout(width = '100px'))
])

In [754]:
boothFuncBtn = ToggleButton(
    description = 'Wykres zależności wartości funkcji celu od kolejnej iteracji',
    disabled = False,
    layout = Layout(width = '50%')
)
meanBoothFuncBtn = ToggleButton(
    description = 'Wykres zależności średniej wartości funkcji celu od kolejnej iteracji',
    disabled = False,
    layout = Layout(width = '50%')
)
stdBtn = ToggleButton(
    description = 'Wykres zależności odchylenia standardowego od kolejnej iteracji',
    disabled = False,
    layout = Layout(width = '50%')
)

In [755]:
plotsBox = Box(
    children = [boothFuncBtn, meanBoothFuncBtn, stdBtn], 
    layout = Layout(display = 'flex', flex_flow = 'column', margin = "10px")
)

In [756]:
fileInput = HBox([
    Label(
        'Zapis wyników do pliku:', 
        layout = Layout(width = '200px')), 
    Text(
        value = '',
        placeholder = 'lokalizacja pliku',
        disabled = False,
        layout = Layout(width = '80%')
    )
])

fileButton = ToggleButton(
    description = 'Zapisz',
    disabled = False,
    layout = Layout(width = '11%')
)

In [757]:
fileBox = Box(
    children = [fileInput, fileButton], 
    layout = Layout(display = 'flex', flex_flow = 'row', margin = "10px")
)

In [758]:
outputBox = Box(
    children = [timeOutput, plotsBox, fileBox], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

In [759]:
outputBox

Box(children=(HBox(children=(Label(value='Czas obliczeń: ', layout=Layout(width='100px')), Label(value='00.00 …